### Find solved posts

In [2]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

period = 'all'
host = "https://discuss.pytorch.org"
total_pages = 1236

solved_post_list = []

def fetch_page(page):
    resp = requests.get(host+f'/top.json?page={page}&period={period}&per_page=100').json()
    df = pd.DataFrame(resp["topic_list"]["topics"])
    df = df.loc[df.has_accepted_answer == True, ["id", "title"]]
    return df

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(fetch_page, page) for page in range(total_pages)]
    for future in tqdm(as_completed(futures), total=total_pages):
        solved_post_list.append(future.result())


100%|██████████| 1236/1236 [15:50<00:00,  1.30it/s]


In [4]:
pd.concat(solved_post_list, ignore_index=True).to_csv("discuss_solved_id.csv", index=False)

### Extract post contents

In [7]:
solved_ids_df = pd.read_csv("discuss_solved_id.csv")

solved_ids_df

,id,title
0,3663,Torch.from_numpy not support negative strides
1,14944,What is the difference between BCEWithLogitsLo...
2,18689,Beginner: Should ReLU/sigmoid be called in the...
3,1041,How to change the default device of GPU? devic...
4,2152,How to train the network with multiple branches
...,...,...
13283,176713,Best way to implement series of weighted rando...
13284,177807,Batch_norm causes RuntimeError: Expected all t...
13285,178060,Scale gradient without scaling loss values
13286,177926,Transform normalization


In [31]:
solved_ids_df.loc[1, "title"]

'What is the difference between BCEWithLogitsLoss and MultiLabelSoftMarginLoss'

In [25]:
title = "14944"

In [26]:
def _process_cooked(cooked):
    bs = BSHTML(cooked)
    p = ' '.join([x.get_text() for x in bs.find_all('p')])
    return p

In [27]:
r = requests.get(host+f'/t/{title}/posts.json').json()

In [28]:
from bs4 import BeautifulSoup as BSHTML
q = title + '? ' + _process_cooked(r['post_stream']['posts'][0]['cooked'])

q

'14944? I think there is no difference between BCEWithLogitsLoss and MultiLabelSoftMarginLoss. BCEWithLogitsLoss = One Sigmoid Layer + BCELoss (solved numerically unstable problem) MultiLabelSoftMargin’s fomula is also same with BCEWithLogitsLoss. One difference is BCEWithLogitsLoss has a ‘weight’ parameter, MultiLabelSoftMarginLoss no has) BCEWithLogitsLoss  :\n MultiLabelSoftMarginLoss  :\n The two formula is exactly the same except for the weight value.'

In [29]:
a = _process_cooked([x['cooked'] for x in r['post_stream']['posts'] if x['accepted_answer'] is True][0])

a

'You are right. Both loss functions seem to return the same loss values:'